In [ ]:
#Import the necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import SGD  # Stochastic Gradient Descent
from tensorflow.keras.optimizers import Adam  # Adam
import numpy as np
import pandas as pd

In [ ]:
#Load the preprocessed data
X_train=pd.read_csv('C:/Users/sarho66/OneDrive - Linköpings universitet/mimic-iii/data/X_train_imputed.csv')
y_train=pd.read_csv('C:/Users/sarho66/OneDrive - Linköpings universitet/mimic-iii/data/y_train.csv')
X_test=pd.read_csv('C:/Users/sarho66/OneDrive - Linköpings universitet/mimic-iii/data/X_test_imputed.csv')
y_test=pd.read_csv('C:/Users/sarho66/OneDrive - Linköpings universitet/mimic-iii/data/y_test.csv')

In [ ]:
# Convert a single-column DataFrame to a Series
y_train = y_train.squeeze()  # If y_train is a DataFrame with one column
y_test = y_test.squeeze()  # If y_test is a DataFrame with one column

In [ ]:
y_train[y_train==1].shape[0]/y_train.shape[0]*100
y_test[y_test==1].shape[0]/y_test.shape[0]*100
print(f" Dead percentage in Train: {y_train[y_train==1].shape[0]/y_train.shape[0]*100:.2f}%, \n Dead percentage in Test: {y_test[y_test==1].shape[0]/y_test.shape[0]*100:.2f}%")

In [ ]:
#Define the model

tf.random.set_seed(123)
model_1 = Sequential([
    # First hidden layer (acts as the input layer with input_shape defined) with 64 neurons
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    # Second hidden layer with 32 neurons
    Dense(32, activation='relu'),
    # Output layer with 1 neuron for binary classification
    Dense(1, activation='sigmoid')
])

In [ ]:
model_1 = Sequential([
    # First hidden layer (acts as the input layer with input_shape defined) with 64 neurons
    Dense(1, activation='sigmoid', input_shape=(X_train.shape[1],)),
])

In [ ]:
model_1.summary()

In [ ]:
#Compiling the model


def custom_loss(y_true, y_pred):
    # Define a small epsilon value
    epsilon = 1e-7
    
    # Add epsilon to y_pred to ensure it's never exactly 0 or 1
    y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)
    
    # Log the minimum values in y_pred to check for zeros or near-zero values
    tf.print("Min value in y_pred:", tf.reduce_min(y_pred))
    tf.print("Min value in y_true:", tf.reduce_min(y_true))

    # Compute the loss as usual
    loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)

    # Check if the loss becomes NaN
    loss_is_nan = tf.reduce_any(tf.math.is_nan(loss))
    if loss_is_nan:
        tf.print("NaN detected in loss")

    return loss



In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) 

model_1.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
#Training the model
history=model_1.fit(X_train,y_train,epochs=150,verbose=1, validation_split=0.2)


In [ ]:
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt

# Assuming 'history' is the object returned by the 'fit' method of your model
# and contains the training history. Remember, indexing starts at 0,
# so 'after 5 epochs' means starting from index 5 (i.e., epoch 6 onwards).

plt.figure(figsize=(14, 5))

# Plotting training & validation accuracy values starting after epoch 5
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])  # Start from index 5 to skip first 5 epochs
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plotting training & validation loss values starting after epoch 5
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()


In [ ]:
#Evaluation of the model
test_loss, test_acc= model_1.evaluate(X_test, y_test)
print(f"Test Acuuracy: {test_acc}")

In [ ]:
predictions=model_1.predict(X_test)